# Novel Writer Studio - Web UI (Colab)

Run a full story-writing web UI on Google Colab.

**Two modes:**
- **Local Model** (GPU required): Your fine-tuned LoRA model writes chapters with trained literary style. Needs Colab Pro for 32B model (A100), free tier works for 8B (T4).
- **Cloud API** (no GPU needed): Gemini/GPT writes chapters. Works on **free Colab** — no GPU runtime required. Loses LoRA style but keeps all agent features.

**Workflow:**
1. **Cloud AI** (Gemini / GPT) develops your story idea into a detailed plot outline
2. **Multi-agent system** (Mastermind + Story Tracker) orchestrates chapter generation
3. **Your chosen writer** (local model or cloud API) generates the chapter prose

### How to use
1. Run cells in order (skip cells 3-4 if using Cloud API mode only)
2. Click the Gradio public link to open the UI
3. Enter your API key, pick your Writer Mode, and generate!

In [ ]:
#@title Configuration { display-mode: "form" }

#@markdown ### Writer Mode
#@markdown - **local_model**: Use your fine-tuned LoRA model (requires GPU runtime)
#@markdown - **cloud_api**: Use Gemini/GPT for writing (no GPU needed, skip cells 3-4)
WRITER_MODE = "cloud_api" #@param ["local_model", "cloud_api"]

#@markdown ---
#@markdown ### Model Selection (only for local_model mode)
#@markdown Choose the base model that matches your LoRA adapter.
MODEL_CHOICE = "qwen3_32b" #@param ["qwen3_4b", "qwen3_8b", "qwen3_14b", "qwen3_32b", "llama31_8b", "gemma2_9b", "mistral_nemo_12b"]

#@markdown ### LoRA Upload Mode
#@markdown - **huggingface_hub**: Download from Hugging Face (recommended)
#@markdown - **upload_zip**: Upload your LoRA adapter as a .zip file
#@markdown - **google_drive**: Load from Google Drive path
LORA_MODE = "huggingface_hub" #@param ["huggingface_hub", "upload_zip", "google_drive"]

#@markdown ### Hugging Face repo ID (only if LORA_MODE = huggingface_hub)
HF_REPO_ID = "YOUR_USER/qwen3_32b_novel_lora" #@param {type:"string"}

#@markdown ### Google Drive path (only if LORA_MODE = google_drive)
DRIVE_LORA_PATH = "/content/drive/MyDrive/qwen3_32b_novel_lora" #@param {type:"string"}

MODEL_CONFIGS = {
    'qwen3_4b': 'unsloth/Qwen3-4B',
    'qwen3_8b': 'unsloth/Qwen3-8B',
    'qwen3_14b': 'unsloth/Qwen3-14B',
    'qwen3_32b': 'unsloth/Qwen3-32B',
    'llama31_8b': 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit',
    'gemma2_9b': 'unsloth/gemma-2-9b-it-bnb-4bit',
    'mistral_nemo_12b': 'unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit',
}

BASE_MODEL = MODEL_CONFIGS[MODEL_CHOICE]

print(f'Writer mode: {WRITER_MODE}')
if WRITER_MODE == 'local_model':
    print(f'Base model: {BASE_MODEL}')
    print(f'LoRA mode: {LORA_MODE}')
else:
    print('Cloud API mode — no GPU or local model needed.')
    print('You can skip cells 3 and 4, go straight to cell 5 (Install deps) then cell 6 (Launch UI).')

In [ ]:
#@title Install dependencies
import subprocess, sys

# Core deps (always needed)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'gradio', 'google-genai', 'openai', 'huggingface_hub'])

if WRITER_MODE == 'local_model':
    # GPU deps
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'unsloth'])
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '--force-reinstall',
                           '--no-cache-dir', '--no-deps',
                           'git+https://github.com/unslothai/unsloth.git'])
    import torch
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('Cloud API mode — GPU dependencies skipped.')

print('Setup complete!')

In [ ]:
#@title Upload / locate LoRA adapter (skip if Cloud API mode)
import os, zipfile
from pathlib import Path

LORA_PATH = None

if WRITER_MODE == 'cloud_api':
    print('Cloud API mode — no LoRA adapter needed. Skip this cell.')

elif LORA_MODE == 'huggingface_hub':
    from huggingface_hub import snapshot_download
    print(f'Downloading LoRA from Hugging Face: {HF_REPO_ID}')
    LORA_PATH = snapshot_download(
        repo_id=HF_REPO_ID,
        local_dir=f'/content/{HF_REPO_ID.split("/")[-1]}',
    )
    print(f'Downloaded to: {LORA_PATH}')

elif LORA_MODE == 'upload_zip':
    from google.colab import files as colab_files
    print('Upload your LoRA adapter zip file:')
    uploaded = colab_files.upload()
    for name in uploaded:
        if name.endswith('.zip'):
            with zipfile.ZipFile(name, 'r') as z:
                z.extractall('/content/')
            for d in Path('/content').iterdir():
                if d.is_dir() and (d / 'adapter_config.json').exists():
                    LORA_PATH = str(d)
                    break
        else:
            os.makedirs('/content/lora_adapter', exist_ok=True)
            os.rename(name, f'/content/lora_adapter/{name}')
            LORA_PATH = '/content/lora_adapter'

elif LORA_MODE == 'google_drive':
    from google.colab import drive
    drive.mount('/content/drive')
    LORA_PATH = DRIVE_LORA_PATH

if LORA_PATH and Path(LORA_PATH).exists():
    print(f'LoRA adapter found: {LORA_PATH}')
    for f in sorted(Path(LORA_PATH).iterdir()):
        print(f'  {f.name} ({f.stat().st_size / 1024:.0f} KB)')
elif WRITER_MODE == 'local_model':
    print(f'ERROR: LoRA adapter not found at {LORA_PATH}')

In [ ]:
#@title Load model + LoRA (skip if Cloud API mode)

model = None
tokenizer = None

if WRITER_MODE == 'cloud_api':
    print('Cloud API mode — no model to load. Skip this cell.')
else:
    from unsloth import FastLanguageModel
    import torch

    print(f'Loading base model: {BASE_MODEL}')
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL,
        max_seq_length=4096,
        dtype=None,
        load_in_4bit=True,
    )

    print(f'Applying LoRA from: {LORA_PATH}')
    from peft import PeftModel
    model = PeftModel.from_pretrained(model, LORA_PATH)
    FastLanguageModel.for_inference(model)

    vram = torch.cuda.memory_allocated() / 1e9
    print(f'\nModel loaded! VRAM used: {vram:.1f} GB')

In [ ]:
#@title Launch Web UI
import gradio as gr
import copy, re, time, json, random
from pathlib import Path

# ---- System prompts (matching training) ----
ZH_SYSTEM = (
    '你是一位经验丰富的中文小说作家，擅长构建沉浸式的叙事场景。请根据给定的上下文续写故事，要求：\n'
    '1. 保持与原文一致的叙事视角和文风\n'
    '2. 通过具体的动作、对话和环境描写推动情节发展\n'
    '3. 角色的言行应符合其性格特征和当前情境\n'
    '4. 善用感官细节（视觉、听觉、触觉、嗅觉）营造氛围\n'
    '5. 对话要自然生动，符合角色身份和说话习惯\n'
    '6. 避免空洞的心理独白，用行动和细节展现人物内心'
)

EN_SYSTEM = (
    'You are an accomplished fiction author with a gift for immersive storytelling. '
    'Continue the narrative following these principles:\n'
    '1. Maintain the established point of view, voice, and tonal register\n'
    '2. Advance the plot through concrete action, dialogue, and environmental detail\n'
    '3. Show character emotion through behavior, body language, and subtext — not exposition\n'
    '4. Engage multiple senses (sight, sound, touch, smell, taste) to ground scenes\n'
    '5. Write dialogue that reveals character, creates tension, and sounds natural\n'
    '6. Vary sentence rhythm — mix short punchy lines with longer flowing passages'
)

# ---- Instruction pools (from training data) ----
_ZH_INSTRUCTIONS = [
    '续写这段叙事，保持原文的风格和节奏。',
    '以相同的文风继续这个故事。',
    '根据已有的情节和人物设定，续写下一段。',
    '保持叙事视角不变，继续推进故事发展。',
    '用生动的细节描写续写这个场景。',
    '通过对话和动作描写推进下面的情节。',
    '延续当前的叙事氛围，写出接下来发生的事。',
    '以细腻的笔触续写这段文字。',
    '按照原文的叙事节奏，写出故事的下一部分。',
    '继续描绘这个场景中的人物和事件。',
    '用符合原文风格的语言续写故事。',
    '展开叙述，让故事自然地向前发展。',
    '保持文风一致，续写接下来的情节。',
    '以沉浸式的叙事方式继续这段故事。',
    '描绘接下来的场景，注意环境和人物的刻画。',
    '用简洁有力的文字续写这段叙事。',
    '继续讲述这个故事，注意情感的表达。',
    '以自然流畅的文笔续写下一段。',
    '延续原文的基调，推进故事走向。',
    '用丰富的感官描写续写这个场景。',
]

_EN_INSTRUCTIONS = [
    'Continue the narrative in the established style.',
    'Write the next passage, maintaining the existing voice and tone.',
    'Advance the story using vivid sensory details.',
    'Continue this scene with natural dialogue and action.',
    'Extend the narrative, preserving the point of view and pacing.',
    'Write what happens next, staying true to the characters.',
    'Continue the story with concrete, immersive description.',
    'Carry the narrative forward in the same literary register.',
    'Write the next segment, matching the established rhythm.',
    'Develop this scene further with authentic detail.',
    'Push the story forward through action and dialogue.',
    'Continue in the same voice, advancing the plot naturally.',
    'Write the following passage in the style of the preceding text.',
    'Extend this scene with attention to atmosphere and character.',
    'Continue the narrative arc with engaging prose.',
    'Write what comes next, maintaining tension and pacing.',
    'Advance the story, weaving in environmental detail.',
    'Continue with prose that matches the tone and texture of the original.',
    'Develop the next beat of the story with precise language.',
    'Carry the scene forward, balancing action with description.',
]

# ---- Token utilities ----
MAX_SEQ_LENGTH = 4096


def count_tokens(text):
    if tokenizer is not None:
        return len(tokenizer.encode(text, add_special_tokens=False))
    # Rough estimate for cloud-only mode (no tokenizer loaded)
    cjk = sum(1 for c in text if '\u4e00' <= c <= '\u9fff')
    return cjk + len(text.split()) - cjk // 2


def _find_sentence_boundary(text, max_chars, from_end=False):
    sentence_ends_cjk = re.compile(r'[。！？…]+')
    sentence_ends_en = re.compile(r'[.!?][\u201c\u201d\u2018\u2019"\')\u300d\uff09]*(?:\s|\n)')
    if from_end:
        search_region = text[-max_chars:] if len(text) > max_chars else text
        offset = max(0, len(text) - max_chars)
        best = 0
        for m in sentence_ends_cjk.finditer(search_region):
            best = m.end()
            break
        for m in sentence_ends_en.finditer(search_region):
            if m.end() < best or best == 0:
                best = m.end()
            break
        return offset + best if best > 0 else offset
    else:
        search_region = text[:max_chars]
        best = max_chars
        for m in sentence_ends_cjk.finditer(search_region):
            best = m.end()
        for m in sentence_ends_en.finditer(search_region):
            best = m.end()
        return best


def trim_to_token_budget(text, max_tokens, keep_end=True):
    current = count_tokens(text)
    if current <= max_tokens:
        return text
    ratio = max_tokens / max(current, 1)
    target_chars = int(len(text) * ratio * 0.95)
    if keep_end:
        start = _find_sentence_boundary(text, len(text) - target_chars, from_end=True)
        trimmed = text[start:]
    else:
        end = _find_sentence_boundary(text, target_chars, from_end=False)
        trimmed = text[:end]
    if count_tokens(trimmed) > max_tokens:
        trimmed = text[-target_chars:] if keep_end else text[:target_chars]
    return trimmed.strip()


def detect_language(text):
    cjk = sum(1 for c in text[:300] if '\u4e00' <= c <= '\u9fff')
    return 'zh' if cjk > len(text[:300]) * 0.15 else 'en'


# ---- Cloud API helpers ----
def _build_plot_prompt(idea, num_chapters, lang):
    if lang == 'zh':
        system = (
            '你是一位资深的小说策划编辑和故事架构师。你擅长从简单的故事构思中发展出完整、'
            '引人入胜的小说大纲。你的大纲应该包含极其丰富的细节，足以直接指导AI模型逐章生成高质量的小说内容。'
            '每个章节的大纲都应该详细到可以独立作为写作指南。'
        )
        prompt = f'请基于以下故事构思，创作一个非常详细的小说大纲：\n\n故事构思：{idea}\n\n请严格按照以下格式输出（使用中文）：\n\n## 小说标题\n[一个引人入胜的标题]\n\n## 故事背景\n[详细的世界观和背景设定，至少200字。包括：时代背景、地理环境、社会体制、文化风俗、特殊设定]\n\n## 主要人物\n（至少4个主要角色，每个角色需包含详细信息）\n- **[角色全名]**（[年龄/外貌简述]）：[性格特点——至少3个性格关键词]，[身份背景]，[核心动机]，[人物弧光]，[与其他角色的关键关系]\n\n## 核心冲突\n[故事的主要矛盾和驱动力，包括外部冲突和内部冲突，至少100字]\n\n## 章节大纲\n（共{num_chapters}章，每章需要非常具体的情节描述）\n'
        for i in range(1, num_chapters + 1):
            prompt += f'\n### 第{i}章：[章节标题]\n- **开场场景**：[具体的时间、地点、氛围描写]\n- **主要事件**：[本章发生的1-3个关键事件]\n- **人物互动**：[哪些角色出场，对话和冲突要点]\n- **情感节奏**：[情感基调变化]\n- **关键细节**：[需要着重描写的场景细节]\n- **章末转折**：[悬念、伏笔或转折点]\n'
        prompt += '\n## 伏笔与线索\n[列出3-5个贯穿全文的伏笔和线索]\n\n## 写作风格指导\n[对本小说整体风格的建议]\n\n请确保章节之间有清晰的因果关系，整体故事有完整的起承转合，每章大纲足够详细。'
    else:
        system = (
            'You are a senior fiction editor and story architect. You excel at developing '
            'simple story concepts into complete, compelling novel outlines with rich detail.'
        )
        prompt = f'Based on the following story idea, create a very detailed novel outline:\n\nStory idea: {idea}\n\n## Title\n[A compelling title]\n\n## Setting\n[Detailed world-building, at least 200 words]\n\n## Main Characters\n(At least 4, each with detailed profiles)\n\n## Central Conflict\n[Main tension, at least 100 words]\n\n## Chapter Outline\n({num_chapters} chapters, each with specific plot details)\n'
        for i in range(1, num_chapters + 1):
            prompt += f'\n### Chapter {i}: [Title]\n- **Opening scene**: [Time, place, atmosphere]\n- **Key events**: [1-3 major events]\n- **Character interactions**: [Dialogue and conflict points]\n- **Emotional rhythm**: [Tone arc]\n- **Key details**: [Important elements to emphasize]\n- **Chapter-end hook**: [Cliffhanger or turning point]\n'
        prompt += '\n## Foreshadowing & Threads\n[3-5 narrative threads]\n\n## Style Guide\n[Recommendations for style]\n\nEnsure clear progression and complete narrative arc.'
    return system, prompt


def generate_plot_gemini(api_key, idea, num_chapters, lang):
    from google import genai
    from google.genai import types
    client = genai.Client(api_key=api_key)
    system, prompt = _build_plot_prompt(idea, num_chapters, lang)
    response = client.models.generate_content(
        model='gemini-3-pro-preview', contents=prompt,
        config=types.GenerateContentConfig(system_instruction=system, temperature=0.9, max_output_tokens=8192),
    )
    return response.text


def generate_plot_gpt(api_key, idea, num_chapters, lang):
    from openai import OpenAI
    client = OpenAI(api_key=api_key)
    system, prompt = _build_plot_prompt(idea, num_chapters, lang)
    response = client.chat.completions.create(
        model='gpt-4o', messages=[{'role': 'system', 'content': system}, {'role': 'user', 'content': prompt}],
        temperature=0.9, max_tokens=8192,
    )
    return response.choices[0].message.content


def develop_plot_api(idea, num_chapters, provider, api_key):
    if not idea.strip():
        return 'Please enter a story idea first.'
    if not api_key.strip():
        return f'Please enter your {provider} API key above.'
    lang = detect_language(idea)
    try:
        if provider == 'Gemini':
            return generate_plot_gemini(api_key, idea, int(num_chapters), lang)
        else:
            return generate_plot_gpt(api_key, idea, int(num_chapters), lang)
    except Exception as e:
        return f'API Error ({provider}): {e}'


def call_cloud_api(system, prompt, provider, api_key, temperature=0.7, max_tokens=2048):
    if provider == 'Gemini':
        from google import genai
        from google.genai import types
        client = genai.Client(api_key=api_key)
        response = client.models.generate_content(
            model='gemini-3-pro-preview', contents=prompt,
            config=types.GenerateContentConfig(
                system_instruction=system, temperature=temperature, max_output_tokens=max_tokens),
        )
        return response.text
    elif provider == 'GPT':
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'system', 'content': system}, {'role': 'user', 'content': prompt}],
            temperature=temperature, max_tokens=max_tokens,
        )
        return response.choices[0].message.content
    else:
        raise ValueError(f'Unknown provider: {provider}')


def _parse_json_response(text):
    m = re.search(r'```(?:json)?\s*\n(.*?)\n```', text, re.DOTALL)
    if m:
        return json.loads(m.group(1))
    cleaned = text.strip()
    if cleaned.startswith('{'):
        return json.loads(cleaned)
    start = cleaned.find('{')
    end = cleaned.rfind('}')
    if start != -1 and end != -1:
        return json.loads(cleaned[start:end + 1])
    raise ValueError(f'Could not parse JSON from response: {text[:200]}...')


# ---- Cloud text generation ----
def generate_text_cloud(prompt, system_prompt='', max_new_tokens=2048, temperature=0.8,
                        provider='Gemini', api_key=''):
    """Generate text using cloud API instead of a local model."""
    if not api_key.strip():
        return 'Please enter your API key in Settings above.'
    if not system_prompt:
        system_prompt = ZH_SYSTEM if detect_language(prompt) == 'zh' else EN_SYSTEM
    return call_cloud_api(system_prompt, prompt, provider, api_key,
                          temperature=temperature, max_tokens=max_new_tokens)


# ---- Local model generation ----
def generate_text(prompt, system_prompt='', max_new_tokens=2048, temperature=0.8,
                  top_p=0.9, top_k=50, repetition_penalty=1.0):
    if model is None or tokenizer is None:
        return 'No local model loaded. Switch Writer Mode to Cloud API or load a model first.'

    if not system_prompt:
        system_prompt = ZH_SYSTEM if detect_language(prompt) == 'zh' else EN_SYSTEM

    import torch
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': prompt},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=max_new_tokens,
            temperature=max(temperature, 0.01), top_p=top_p, top_k=top_k,
            repetition_penalty=repetition_penalty, do_sample=True,
        )
    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True)


# ---- Story Tracker ----
class StoryTracker:
    @staticmethod
    def empty_bible():
        return {'characters': {}, 'plot_threads': [], 'chapter_summaries': [], 'style_notes': ''}

    @staticmethod
    def initialize_from_outline(outline, provider, api_key, lang):
        if lang == 'zh':
            system = '你是一位小说编辑助手。请从小说大纲中提取结构化信息，以JSON格式输出。'
            prompt = f'请分析以下小说大纲，提取关键信息并以JSON格式输出：\n\n{outline}\n\n输出格式（必须是合法JSON）：\n```json\n{{"characters": {{"角色名": {{"description": "外貌和身份", "personality": "性格", "motivation": "动机", "relationships": {{}}, "location": "位置", "emotional_state": "状态"}}}}, "plot_threads": [{{"name": "线索", "status": "active", "description": "描述"}}], "style_notes": "风格"}}\n```'
        else:
            system = 'You are a fiction editor assistant. Extract structured information from the novel outline as JSON.'
            prompt = f'Analyze the following novel outline and extract key information as JSON:\n\n{outline}\n\nOutput format (must be valid JSON):\n```json\n{{"characters": {{"Name": {{"description": "role", "personality": "traits", "motivation": "motivation", "relationships": {{}}, "location": "location", "emotional_state": "state"}}}}, "plot_threads": [{{"name": "thread", "status": "active", "description": "desc"}}], "style_notes": "style"}}\n```'
        try:
            response = call_cloud_api(system, prompt, provider, api_key, temperature=0.3)
            parsed = _parse_json_response(response)
            bible = StoryTracker.empty_bible()
            bible['characters'] = parsed.get('characters', {})
            bible['plot_threads'] = parsed.get('plot_threads', [])
            bible['style_notes'] = parsed.get('style_notes', '')
            return bible
        except Exception as e:
            bible = StoryTracker.empty_bible()
            bible['style_notes'] = f'(Bible init failed: {e})'
            return bible

    @staticmethod
    def apply_updates(bible, updates):
        bible = copy.deepcopy(bible)
        for name, changes in updates.get('characters', {}).items():
            if name in bible['characters']:
                bible['characters'][name].update(changes)
            else:
                bible['characters'][name] = changes
        existing_names = {t['name'] for t in bible['plot_threads']}
        for thread in updates.get('plot_threads', []):
            if thread['name'] in existing_names:
                for i, t in enumerate(bible['plot_threads']):
                    if t['name'] == thread['name']:
                        bible['plot_threads'][i].update(thread)
                        break
            else:
                bible['plot_threads'].append(thread)
        if 'chapter_summary' in updates:
            bible.setdefault('chapter_summaries', []).append(updates['chapter_summary'])
        return bible

    @staticmethod
    def get_context_summary(bible, ch_num, lang):
        if not bible or not bible.get('characters'):
            return ''
        parts = []
        if lang == 'zh':
            chars = []
            for name, info in bible.get('characters', {}).items():
                loc = info.get('location', '')
                emo = info.get('emotional_state', '')
                chars.append(f'{name}：{loc}，{emo}')
            if chars:
                parts.append('【人物状态】' + '；'.join(chars))
            active = [t for t in bible.get('plot_threads', []) if t.get('status') == 'active']
            if active:
                parts.append(f"【活跃线索】{'、'.join(t['name'] for t in active[:5])}")
            for s in bible.get('chapter_summaries', [])[-3:]:
                parts.append(f"【第{s.get('chapter', '?')}章】{s.get('summary', '')}")
        else:
            chars = []
            for name, info in bible.get('characters', {}).items():
                loc = info.get('location', '')
                emo = info.get('emotional_state', '')
                chars.append(f'{name}: {loc}, {emo}')
            if chars:
                parts.append('[Characters] ' + '; '.join(chars))
            active = [t for t in bible.get('plot_threads', []) if t.get('status') == 'active']
            if active:
                parts.append(f"[Active threads] {', '.join(t['name'] for t in active[:5])}")
            for s in bible.get('chapter_summaries', [])[-3:]:
                parts.append(f"[Ch.{s.get('chapter', '?')}] {s.get('summary', '')}")
        return '\n'.join(parts)


# ---- Mastermind ----
class Mastermind:
    @staticmethod
    def _extract_chapter_outline(outline, ch_num):
        pattern = rf'(?:###?\s*(?:第{ch_num}章|Chapter\s*{ch_num})[：:\s]*)(.+?)(?=(?:###?\s*(?:第\d+章|Chapter\s*\d+))|$)'
        match = re.search(pattern, outline, re.DOTALL)
        return match.group(0).strip() if match else f'Chapter {ch_num}'

    @staticmethod
    def plan_chapter(outline, ch_num, bible, prev_ending, provider, api_key, lang):
        chapter_outline = Mastermind._extract_chapter_outline(outline, ch_num)
        bible_summary = StoryTracker.get_context_summary(bible, ch_num, lang)
        if lang == 'zh':
            system = '你是一位资深小说编辑。你的任务是将结构化的章节大纲转化为散文体的场景引子，供AI写手作为写作起点。场景引子必须是纯散文，不能包含任何标题、列表、或Markdown格式。'
            prompt = f'请为第{ch_num}章创建写作计划。\n\n【本章大纲】\n{chapter_outline}\n\n【故事圣经】\n{bible_summary if bible_summary else "（首章，无历史记录）"}\n\n【上一章结尾】\n{prev_ending[-800:] if prev_ending else "（首章）"}\n\n请输出JSON格式（必须合法JSON）：\n```json\n{{"scene_primer": "用散文体写一段场景引子（200-400字），描述本章开场的环境、氛围和人物状态。必须是纯叙事散文。", "key_events": ["事件1", "事件2", "事件3"], "guidance": "写作指导"}}\n```'
        else:
            system = 'You are a senior fiction editor. Convert structured chapter outlines into prose scene primers for an AI writer. The scene primer MUST be pure prose — no headings, bullet points, or markdown.'
            prompt = f'Create a writing plan for Chapter {ch_num}.\n\n[Chapter outline]\n{chapter_outline}\n\n[Story bible]\n{bible_summary if bible_summary else "(First chapter, no history)"}\n\n[End of previous chapter]\n{prev_ending[-800:] if prev_ending else "(First chapter)"}\n\nOutput as JSON (must be valid JSON):\n```json\n{{"scene_primer": "Write a prose scene primer (150-300 words). Must be pure narrative prose.", "key_events": ["Event 1", "Event 2", "Event 3"], "guidance": "Writing guidance"}}\n```'
        try:
            response = call_cloud_api(system, prompt, provider, api_key, temperature=0.7)
            plan = _parse_json_response(response)
            plan.setdefault('scene_primer', chapter_outline)
            plan.setdefault('key_events', [])
            plan.setdefault('guidance', '')
            return plan
        except Exception as e:
            return {'scene_primer': chapter_outline, 'key_events': [], 'guidance': f'(Planning failed: {e})'}

    @staticmethod
    def review_and_update(plan, text, bible, ch_num, provider, api_key, lang):
        key_events = ', '.join(plan.get('key_events', []))
        bible_summary = StoryTracker.get_context_summary(bible, ch_num, lang)
        if lang == 'zh':
            system = '你同时扮演两个角色：1）小说编辑——审核章节质量；2）故事记录员——更新故事圣经。请以JSON格式输出。'
            prompt = f'请审核以下生成的第{ch_num}章，并更新故事圣经。\n\n【写作计划的关键事件】\n{key_events}\n\n【生成的章节文本】（前2000字）\n{text[:3000]}\n\n【当前故事圣经】\n{bible_summary if bible_summary else "（空）"}\n\n请输出JSON格式：\n```json\n{{"review": {{"approved": true, "scores": {{"plot_adherence": 8, "prose_quality": 7, "character_consistency": 8, "engagement": 7}}, "issues": [], "regen_guidance": ""}}, "bible_updates": {{"characters": {{}}, "plot_threads": [], "chapter_summary": {{"chapter": {ch_num}, "summary": "概要"}}}}}}\n```'
        else:
            system = 'You play two roles: 1) Fiction editor — review chapter quality; 2) Story recorder — update the story bible. Output as JSON.'
            prompt = f'Review Chapter {ch_num} and update the story bible.\n\n[Key events]\n{key_events}\n\n[Generated text] (first 2000 words)\n{text[:3000]}\n\n[Current bible]\n{bible_summary if bible_summary else "(empty)"}\n\nOutput as JSON:\n```json\n{{"review": {{"approved": true, "scores": {{"plot_adherence": 8, "prose_quality": 7, "character_consistency": 8, "engagement": 7}}, "issues": [], "regen_guidance": ""}}, "bible_updates": {{"characters": {{}}, "plot_threads": [], "chapter_summary": {{"chapter": {ch_num}, "summary": "summary"}}}}}}\n```'
        try:
            response = call_cloud_api(system, prompt, provider, api_key, temperature=0.3)
            result = _parse_json_response(response)
            result.setdefault('review', {'approved': True, 'scores': {}, 'issues': [], 'regen_guidance': ''})
            result.setdefault('bible_updates', {})
            return result
        except Exception as e:
            return {
                'review': {'approved': True, 'scores': {}, 'issues': [f'Review failed: {e}'], 'regen_guidance': ''},
                'bible_updates': {'chapter_summary': {'chapter': ch_num, 'summary': text[:100] + '...'}},
            }


# ---- Prompt Builder ----
class PromptBuilder:
    @staticmethod
    def build_chapter_prompt(instruction, scene_primer, prev_text, max_new_tokens, lang):
        system = ZH_SYSTEM if lang == 'zh' else EN_SYSTEM
        system_tokens = count_tokens(system)
        instruction_tokens = count_tokens(instruction)
        template_overhead = 15
        # Sanitize primer: strip any markdown the cloud API may have included
        scene_primer = re.sub(r'^#+\s.*$', '', scene_primer, flags=re.MULTILINE)
        scene_primer = re.sub(r'^\s*[-*]\s+', '', scene_primer, flags=re.MULTILINE)
        scene_primer = re.sub(r'\*\*([^*]+)\*\*', r'\1', scene_primer)
        scene_primer = scene_primer.strip()
        available = MAX_SEQ_LENGTH - system_tokens - instruction_tokens - template_overhead - max_new_tokens
        available = max(available, 100)
        primer_budget = min(count_tokens(scene_primer), available // 3)
        context_budget = available - primer_budget
        trimmed_primer = trim_to_token_budget(scene_primer, primer_budget, keep_end=False)
        trimmed_context = trim_to_token_budget(prev_text, context_budget, keep_end=True)
        parts = [instruction]
        if trimmed_primer:
            parts.append(trimmed_primer)
        if trimmed_context:
            parts.append(trimmed_context)
        user_content = '\n\n'.join(parts)
        user_tokens = count_tokens(user_content)
        total = system_tokens + user_tokens + template_overhead + max_new_tokens
        return system, user_content, {
            'system_tokens': system_tokens, 'user_tokens': user_tokens,
            'max_new_tokens': max_new_tokens, 'total_estimated': total,
            'within_budget': total <= MAX_SEQ_LENGTH,
        }


# ---- Multi-pass generation ----
def generate_chapter_multipass(scene_primer, prev_text, num_passes, tokens_per_pass,
                                lang, temperature, top_p,
                                use_cloud=False, cloud_provider='', cloud_api_key=''):
    instructions = _ZH_INSTRUCTIONS if lang == 'zh' else _EN_INSTRUCTIONS
    builder = PromptBuilder()
    accumulated = ''
    pass_log = []
    for i in range(num_passes):
        instruction = random.choice(instructions)
        if i == 0:
            context = prev_text[-1200:] if prev_text else ''
            primer = scene_primer
        else:
            context = accumulated[-1000:]
            primer = ''
        system, user_content, token_info = builder.build_chapter_prompt(
            instruction, primer, context, tokens_per_pass, lang)
        if use_cloud:
            result = generate_text_cloud(user_content, system_prompt=system, max_new_tokens=tokens_per_pass,
                                         temperature=temperature, provider=cloud_provider, api_key=cloud_api_key)
        else:
            result = generate_text(user_content, system_prompt=system, max_new_tokens=tokens_per_pass,
                                   temperature=temperature, top_p=top_p, repetition_penalty=1.0)
        accumulated += result
        mode_label = 'cloud' if use_cloud else 'local'
        pass_log.append(f'Pass {i+1}/{num_passes} ({mode_label}): +{len(result)} chars ({token_info["total_estimated"]}/{MAX_SEQ_LENGTH} tokens)')
    return accumulated, pass_log


# ---- Legacy chapter generation ----
def generate_chapter_legacy(plot_outline, chapter_num, previous_text, style_notes,
                            max_tokens, temperature, top_p, rep_penalty):
    lang = detect_language(plot_outline)
    ch = int(chapter_num)
    pattern = rf'(?:###?\s*(?:第{ch}章|Chapter\s*{ch})[：:\s]*)(.+?)(?=(?:###?\s*(?:第\d+章|Chapter\s*\d+))|$)'
    match = re.search(pattern, plot_outline, re.DOTALL)
    chapter_outline = match.group(0).strip() if match else f'Chapter {ch}'
    if lang == 'zh':
        prompt = f'## 小说大纲\n{plot_outline}\n\n'
        if previous_text:
            ctx = previous_text[-2000:] if len(previous_text) > 2000 else previous_text
            prompt += f'## 上一章结尾\n{ctx}\n\n'
        prompt += f'## 当前任务\n请根据以上大纲，撰写第{ch}章的完整内容。\n本章大纲：{chapter_outline}\n\n'
        if style_notes:
            prompt += f'风格要求：{style_notes}\n\n'
        prompt += f'要求：\n1. 以具体的场景描写开头\n2. 通过对话和动作推动情节\n3. 注意人物性格的一致性\n4. 章节结尾要有悬念或转折\n5. 写出完整的章节内容\n\n第{ch}章正文：\n'
    else:
        prompt = f'## Novel Outline\n{plot_outline}\n\n'
        if previous_text:
            ctx = previous_text[-2000:] if len(previous_text) > 2000 else previous_text
            prompt += f'## End of Previous Chapter\n{ctx}\n\n'
        prompt += f'## Current Task\nWrite the complete text of Chapter {ch}.\nChapter outline: {chapter_outline}\n\n'
        if style_notes:
            prompt += f'Style notes: {style_notes}\n\n'
        prompt += f'Requirements:\n1. Open with vivid scene-setting\n2. Drive the plot through dialogue and action\n3. Maintain consistent characterization\n4. End with a hook or turning point\n5. Write the complete chapter\n\nChapter {ch}:\n'
    system = ZH_SYSTEM if lang == 'zh' else EN_SYSTEM
    return generate_text(prompt, system_prompt=system, max_new_tokens=int(max_tokens),
                         temperature=temperature, top_p=top_p, repetition_penalty=rep_penalty)


# ---- Full pipeline ----
def generate_chapter_pipeline(outline, ch_num, chapters_state, style_notes,
                              max_tokens, temperature, top_p, rep_penalty,
                              enable_agents, num_passes, provider, api_key, bible_state,
                              writer_mode='Local Model'):
    use_cloud = writer_mode == 'Cloud API'
    ch_num = int(ch_num)
    lang = detect_language(outline)

    if not use_cloud and model is None:
        msg = "Please load a local model first, or switch Writer Mode to 'Cloud API'."
        return msg, chapters_state, bible_state, msg, ''

    if use_cloud and not api_key.strip():
        msg = 'Cloud writer mode requires an API key. Please enter it in API Settings.'
        return msg, chapters_state, bible_state, msg, ''

    # Legacy mode (local only, no agents)
    if not enable_agents:
        if use_cloud:
            msg = 'Legacy mode requires a local model. Please enable agents for cloud-only writing.'
            return msg, chapters_state, bible_state, msg, ''
        result = generate_chapter_legacy(outline, ch_num, chapters_state, style_notes,
                                         max_tokens, temperature, top_p, rep_penalty)
        sep = f'\n\n{"="*40}\n第{ch_num}章 / Chapter {ch_num}\n{"="*40}\n\n'
        new_acc = chapters_state + sep + result if chapters_state else result
        return result, new_acc, bible_state, 'Legacy mode (agents disabled).', ''

    # Agents require API key
    if not api_key.strip():
        msg = 'Agent mode requires an API key. Please enter it in API Settings.'
        return msg, chapters_state, bible_state, msg, ''

    # Agent mode
    log_lines = []
    def log(msg):
        log_lines.append(f'[{time.strftime("%H:%M:%S")}] {msg}')

    mode_label = 'cloud' if use_cloud else 'local'
    log(f'Writer mode: {mode_label}')

    if not bible_state or not bible_state.get('characters'):
        log('Initializing story bible from outline...')
        bible_state = StoryTracker.initialize_from_outline(outline, provider, api_key, lang)
        log(f'Bible initialized: {len(bible_state.get("characters", {}))} characters')

    log(f'Mastermind planning chapter {ch_num}...')
    prev_ending = chapters_state[-1200:] if chapters_state else ''
    plan = Mastermind.plan_chapter(outline, ch_num, bible_state, prev_ending, provider, api_key, lang)
    log(f'Scene primer: {plan["scene_primer"][:150].replace(chr(10), " ")}...')
    if plan['key_events']:
        log(f'Key events: {", ".join(plan["key_events"][:3])}')

    tokens_per_pass = min(int(max_tokens) // int(num_passes), 1500)
    if tokens_per_pass < 512:
        log(f'Warning: tokens_per_pass ({tokens_per_pass}) below minimum 512, clamping up')
        tokens_per_pass = 512
    log(f'Generating chapter ({int(num_passes)} passes, {tokens_per_pass} tokens/pass, {mode_label})...')
    result, pass_log = generate_chapter_multipass(
        plan['scene_primer'], prev_ending, int(num_passes), tokens_per_pass, lang, temperature, top_p,
        use_cloud=use_cloud, cloud_provider=provider, cloud_api_key=api_key)
    for pl in pass_log:
        log(pl)
    log(f'Total generated: {len(result)} chars')

    log('Reviewing chapter + updating story bible...')
    review_result = Mastermind.review_and_update(plan, result, bible_state, ch_num, provider, api_key, lang)
    review = review_result.get('review', {})
    bible_updates = review_result.get('bible_updates', {})
    bible_state = StoryTracker.apply_updates(bible_state, bible_updates)

    approved = review.get('approved', True)
    scores = review.get('scores', {})
    issues = review.get('issues', [])
    status = 'APPROVED' if approved else 'NEEDS REVISION'
    log(f'Review: {status}')
    if scores:
        log(f'Scores: {", ".join(f"{k}: {v}/10" for k, v in scores.items())}')

    review_text = f'Status: {status}\n'
    if scores:
        review_text += 'Scores:\n' + ''.join(f'  {k}: {v}/10\n' for k, v in scores.items())
    if issues:
        review_text += 'Issues:\n' + ''.join(f'  - {issue}\n' for issue in issues)
    if not approved and review.get('regen_guidance'):
        review_text += f'\nRevision guidance: {review["regen_guidance"]}\n'

    sep = f'\n\n{"="*40}\n第{ch_num}章 / Chapter {ch_num}\n{"="*40}\n\n'
    new_acc = chapters_state + sep + result if chapters_state else result
    return result, new_acc, bible_state, '\n'.join(log_lines), review_text


def continue_writing(existing_text, instruction, max_tokens, temperature, top_p, rep_penalty,
                     writer_mode='Local Model', api_provider='Gemini', api_key=''):
    use_cloud = writer_mode == 'Cloud API'
    if not use_cloud and model is None:
        return "Please load a local model first, or switch Writer Mode to 'Cloud API'."
    if use_cloud and not api_key.strip():
        return 'Cloud writer mode requires an API key. Please enter it in API Settings.'

    lang = detect_language(existing_text)
    if not instruction:
        instruction = '续写这段叙事，保持原文的风格和节奏。' if lang == 'zh' else 'Continue the narrative in the established style.'
    prompt = instruction + '\n\n' + existing_text
    system = ZH_SYSTEM if lang == 'zh' else EN_SYSTEM

    if use_cloud:
        return generate_text_cloud(prompt, system_prompt=system, max_new_tokens=int(max_tokens),
                                   temperature=temperature, provider=api_provider, api_key=api_key)
    else:
        return generate_text(prompt, system_prompt=system, max_new_tokens=int(max_tokens),
                             temperature=temperature, top_p=top_p, repetition_penalty=rep_penalty)


# ---- Determine default writer mode from config ----
_default_writer_mode = 'Cloud API' if WRITER_MODE == 'cloud_api' else 'Local Model'

# ---- Build Gradio UI ----
with gr.Blocks(title='Novel Writer Studio') as app:
    gr.Markdown('# Novel Writer Studio (Colab)')
    gr.Markdown(
        '*Cloud AI develops your plot + orchestrates agents. '
        'Chapters can be written by your fine-tuned local model OR entirely via cloud API (no GPU needed).*'
    )

    # Shared state
    all_chapters = gr.State('')
    bible_state = gr.State({})

    # API Settings
    with gr.Accordion('API Settings', open=True):
        gr.Markdown('Enter your API key. Used for plot generation, agent orchestration, and cloud writing.')
        with gr.Row():
            api_provider = gr.Radio(['Gemini', 'GPT'], value='Gemini', label='Provider')
            api_key_input = gr.Textbox(label='API Key', type='password', placeholder='Paste your API key...', scale=3)

    # Writer Mode
    with gr.Accordion('Writer Mode', open=True):
        gr.Markdown(
            '**Local Model**: Uses your fine-tuned LoRA model (requires GPU). Best quality for style-specific prose.\n\n'
            '**Cloud API**: Uses Gemini/GPT for chapter writing too. No GPU needed. '
            'Loses LoRA style but keeps all agent features (planning, review, story bible).'
        )
        writer_mode = gr.Radio(
            ['Local Model', 'Cloud API'], value=_default_writer_mode,
            label='Chapter Writer', info='Choose what writes the chapter prose')

    # Agent Settings
    with gr.Accordion('Agent Settings', open=False):
        gr.Markdown(
            '**Multi-Agent System**: Mastermind + Story Tracker orchestrate chapter generation.\n'
            '- **Mastermind**: Converts outline to prose scene primers, reviews output\n'
            '- **Story Tracker**: Maintains character states, plot threads, summaries\n'
            '- Uses 2-3 cloud API calls per chapter'
        )
        with gr.Row():
            enable_agents = gr.Checkbox(value=True, label='Enable Agents',
                                        info='Use Mastermind + Story Tracker')
            num_passes = gr.Slider(1, 5, value=3, step=1, label='Generation Passes',
                                   info='More passes = longer chapters (~1000-1500 chars each)')

    # Generation settings
    with gr.Accordion('Chapter Generation Settings', open=False):
        with gr.Row():
            max_tokens_slider = gr.Slider(128, 4096, value=2048, step=128, label='Max New Tokens')
            temperature_slider = gr.Slider(0.1, 2.0, value=0.8, step=0.05, label='Temperature')
        with gr.Row():
            top_p_slider = gr.Slider(0.1, 1.0, value=0.9, step=0.05, label='Top-P')
            rep_penalty_slider = gr.Slider(1.0, 2.0, value=1.0, step=0.05, label='Repetition Penalty',
                                           info='1.0 recommended (matches training)')

    with gr.Tabs():
        # Tab 1: Story Workshop
        with gr.Tab('Story Workshop'):
            gr.Markdown('### Step 1: Your Story Idea')
            with gr.Row():
                with gr.Column(scale=3):
                    story_idea = gr.Textbox(
                        label='Story Idea', lines=4,
                        placeholder='例：在一个武林高手辈出的时代，一个失忆的少年在雪山醒来...\n\nOr: In a world where magic is fueled by music...',
                    )
                with gr.Column(scale=1):
                    num_chapters = gr.Slider(3, 20, value=8, step=1, label='Chapters')
                    develop_btn = gr.Button('Develop Plot (Cloud AI)', variant='primary', size='lg')

            gr.Markdown('### Step 2: Plot Outline')
            gr.Markdown('Generated by Gemini/GPT. Review and edit freely.')
            plot_outline = gr.Textbox(label='Plot Outline (editable)', lines=25,
                                     placeholder='Click Develop Plot to generate...')
            develop_btn.click(develop_plot_api, [story_idea, num_chapters, api_provider, api_key_input], plot_outline)

            gr.Markdown('---')
            gr.Markdown('### Step 3: Generate Chapters')
            gr.Markdown('Writes each chapter using your chosen Writer Mode. Enable agents for multi-pass generation with plot tracking and review.')
            with gr.Row():
                with gr.Column(scale=1):
                    chapter_num = gr.Slider(1, 20, value=1, step=1, label='Chapter Number')
                    style_notes = gr.Textbox(label='Style Notes (optional)', lines=2, placeholder='e.g., 多用对话')
                    gen_chapter_btn = gr.Button('Generate Chapter', variant='primary', size='lg')
                with gr.Column(scale=3):
                    chapter_output = gr.Textbox(label='Generated Chapter', lines=25)

            with gr.Accordion('Generation Log', open=False):
                gen_log_display = gr.Textbox(label='Agent Activity Log', lines=12, interactive=False)

            with gr.Accordion('Chapter Review', open=False):
                review_display = gr.Textbox(label='Mastermind Review', lines=8, interactive=False)

            gen_chapter_btn.click(
                generate_chapter_pipeline,
                [plot_outline, chapter_num, all_chapters, style_notes,
                 max_tokens_slider, temperature_slider, top_p_slider, rep_penalty_slider,
                 enable_agents, num_passes, api_provider, api_key_input, bible_state,
                 writer_mode],
                [chapter_output, all_chapters, bible_state, gen_log_display, review_display],
            )

            with gr.Accordion('All Generated Chapters', open=False):
                all_chapters_display = gr.Textbox(label='Full Story So Far', lines=30, interactive=False)
                refresh_btn = gr.Button('Refresh')
                refresh_btn.click(lambda x: x, all_chapters, all_chapters_display)

                export_btn = gr.Button('Export Story to File')
                export_file = gr.File(label='Download')

                def export_story(text):
                    if not text:
                        return None
                    fpath = f'/content/story_{time.strftime("%Y%m%d_%H%M%S")}.txt'
                    Path(fpath).write_text(text, encoding='utf-8')
                    return fpath

                export_btn.click(export_story, all_chapters, export_file)

        # Tab 2: Story Bible
        with gr.Tab('Story Bible'):
            gr.Markdown('### Story Bible\nTracks character states, plot threads, and chapter summaries.')
            bible_display = gr.JSON(label='Current Story Bible')
            with gr.Row():
                refresh_bible_btn = gr.Button('Refresh Display')
                init_bible_btn = gr.Button('Initialize Bible from Outline', variant='primary')
                clear_bible_btn = gr.Button('Clear Bible', variant='stop')

            bible_manual_notes = gr.Textbox(
                label='Manual Notes (added to style_notes)', lines=3,
                placeholder="Add your own notes (e.g., 'the sword is named Frostbite')")
            add_notes_btn = gr.Button('Add Notes to Bible')
            bible_status = gr.Textbox(label='Status', interactive=False, lines=1)

            def refresh_bible(bible):
                return bible

            def init_bible_from_outline(outline, provider, key, bible):
                if not outline.strip():
                    return bible, bible, 'No outline to initialize from.'
                if not key.strip():
                    return bible, bible, f'Please enter your {provider} API key.'
                lang = detect_language(outline)
                new_bible = StoryTracker.initialize_from_outline(outline, provider, key, lang)
                n_chars = len(new_bible.get('characters', {}))
                return new_bible, new_bible, f'Bible initialized: {n_chars} characters found.'

            def clear_bible():
                return StoryTracker.empty_bible()

            def add_manual_notes(bible, notes):
                if not notes.strip():
                    return bible
                bible = copy.deepcopy(bible)
                existing = bible.get('style_notes', '')
                bible['style_notes'] = (existing + '\n' + notes).strip() if existing else notes
                return bible

            refresh_bible_btn.click(refresh_bible, bible_state, bible_display)
            init_bible_btn.click(init_bible_from_outline,
                                 [plot_outline, api_provider, api_key_input, bible_state],
                                 [bible_state, bible_display, bible_status])
            clear_bible_btn.click(clear_bible, outputs=[bible_state])
            add_notes_btn.click(add_manual_notes, [bible_state, bible_manual_notes], bible_state)

        # Tab 3: Free Write
        with gr.Tab('Free Write'):
            gr.Markdown('### Direct Generation')
            gr.Markdown('Write freely — uses your chosen Writer Mode.')
            with gr.Row():
                with gr.Column():
                    free_context = gr.Textbox(label='Context / Previous Text', lines=10,
                                             placeholder='Paste existing text for continuation...')
                    free_instruction = gr.Textbox(label='Instruction (optional)', lines=2,
                                                 placeholder='e.g., 续写这段战斗场景')
                    free_gen_btn = gr.Button('Generate', variant='primary', size='lg')
                with gr.Column():
                    free_output = gr.Textbox(label='Generated Text', lines=20)
                    append_btn = gr.Button('Append to Context')

            free_gen_btn.click(
                continue_writing,
                [free_context, free_instruction, max_tokens_slider, temperature_slider, top_p_slider, rep_penalty_slider,
                 writer_mode, api_provider, api_key_input],
                free_output,
            )
            append_btn.click(lambda ctx, out: ctx + '\n' + out if ctx else out, [free_context, free_output], free_context)

        # Tab 4: Quick Test
        with gr.Tab('Quick Test'):
            gr.Markdown('### Test with a single prompt')
            gr.Markdown('Uses your chosen Writer Mode.')
            test_prompt = gr.Textbox(label='Prompt', lines=4,
                                    placeholder='月色如霜，照在悬崖边两道对峙的身影上...')
            test_btn = gr.Button('Generate', variant='primary')
            test_output = gr.Textbox(label='Output', lines=15)

            def run_test(prompt, max_t, temp, tp, rp, wmode, provider, key):
                if wmode == 'Cloud API':
                    return generate_text_cloud(prompt, max_new_tokens=int(max_t), temperature=temp,
                                               provider=provider, api_key=key)
                return generate_text(prompt, max_new_tokens=int(max_t), temperature=temp, top_p=tp, repetition_penalty=rp)

            test_btn.click(run_test,
                           [test_prompt, max_tokens_slider, temperature_slider, top_p_slider, rep_penalty_slider,
                            writer_mode, api_provider, api_key_input],
                           test_output)

print('Launching Web UI...')
app.launch(share=True)